In [1]:
import json
import pandas as pd

with open("rankings/heroes.json", "r") as file:
    hero_rankings_by_role = json.load(file)

hero_rankings = {hero.pop("Hero"): hero
                 for heroes in hero_rankings_by_role.values()
                 for hero in heroes}

hero_rankings = pd.DataFrame(hero_rankings).transpose()
hero_rankings

,Mobility,Power,Utility,Range
D.Va,9,7,6,5
Doomfist,10,6,7,3
Junker Queen,6,8,6,4
Orisa,4,7,7,6
Ramattra,5,8,7,5
Reinhardt,4,8,6,2
Roadhog,3,8,5,3
Sigma,5,7,8,7
Winston,9,6,7,5
Wrecking Ball,10,6,8,4


In [2]:
def compute_matchup_score(hero1, hero2):
    hero1_ranking = hero_rankings.loc[hero1]
    hero2_ranking = hero_rankings.loc[hero2]
    matchup = pd.Series()
    for category in hero1_ranking.index:
        if category == "Hero":
            continue
        diff = hero1_ranking.loc[category] - hero2_ranking.loc[category]
        score = diff * abs(diff)
        matchup.loc[category] = score
    matchup.loc["Total"] = matchup.sum()
    matchup.loc["Hero"] = hero2
    return matchup

compute_matchup_score("Winston", "Ana")

Mobility     36
Power        -4
Utility      -4
Range       -25
Total         3
Hero        Ana
dtype: object

In [3]:
def build_matchup_table(hero):
    matchups = []
    for opp in hero_rankings.index:
        if opp == hero:
            continue
        matchups.append(compute_matchup_score(hero, opp))
    df = pd.DataFrame(matchups)
    return df.set_index("Hero").sort_values(by="Total", ascending=False)

build_matchup_table("Winston")

,Mobility,Power,Utility,Range,Total
Hero,,,,,
Roadhog,36,-4,4,4,40
Torbjörn,36,-1,0,-1,34
Reinhardt,25,-4,1,9,31
Cassidy,25,-4,9,-1,29
Bastion,36,-9,4,-4,27
Brigitte,25,-1,0,1,25
Orisa,25,-1,0,-1,23
Zarya,25,-4,1,1,23
Symmetra,25,-1,-1,0,23
